In [1]:
import pandas as pd

In [49]:
original_file = pd.read_csv('log.csv') 

file = original_file.copy()

In [50]:
file.head(3)

,trackerId,task,status,server_region,timestamp,server,state,country,datetime
0,89328463487,Data quality assurance,success,Florida;USA,1599093305000,464f:bdbd:1d:535a:2dad:5117:a92f:b359,Florida,USA,2020-09-03 00:35:05
1,89328463488,Designing and implementing data models,success,Ontario;Canada,1589218599000,1366:c314:8d40:4219:4a51:138f:7ce3:d244,Ontario,Canada,2020-05-11 17:36:39
2,89328463489,Extracting data from various sources,success,California;USA,1595463446000,1ffa:cdd1:7a2b:a7d8:ab00:eada:94b3:7a7c,California,USA,2020-07-23 00:17:26


## 1. Group the data into tasks and count the number of operations per task

In [52]:
by_task = file.groupby(['task']).count()

by_task.head()

,trackerId,status,server_region,timestamp,server,state,country,datetime
task,,,,,,,,
Building data pipelines,102,102,102,102,102,102,102,102
Data governance,116,116,116,116,116,116,116,116
Data integration,100,100,100,100,100,100,100,100
Data quality assurance,111,111,111,111,111,111,111,111
Designing and implementing data models,79,79,79,79,79,79,79,79


## 2. Group by country and count 

In [70]:
# group by country and count 
by_country = file.groupby(['country']).count()

by_country

,trackerId,task,status,server_region,timestamp,server,state,datetime
country,,,,,,,,
Australia,128,128,128,128,128,128,128,128
Canada,124,124,124,124,124,124,124,124
France,124,124,124,124,124,124,124,124
Japan,133,133,133,133,133,133,133,133
USA,491,491,491,491,491,491,491,491


## 3. Group by month and count

In [61]:
file['datetime'] =  pd.to_datetime(file['datetime'])  #convert datetime column to datetime

by_month = file.groupby(pd.Grouper(key='datetime', freq='M')).count() # group by month

by_month.head()

,trackerId,task,status,server_region,timestamp,server,state,country
datetime,,,,,,,,
2020-04-30,124,124,124,124,124,124,124,124
2020-05-31,117,117,117,117,117,117,117,117
2020-06-30,127,127,127,127,127,127,127,127
2020-07-31,103,103,103,103,103,103,103,103
2020-08-31,113,113,113,113,113,113,113,113
